In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2
from db.database import DatabaseConnector
from dtloader.dataloader import DataLoader
from utils.utils import *
from segmentation.segmentation import Segmentation
from sklearn import preprocessing
path = '../stats_files/'

In [ ]:
dt = DataLoader()
dt.load('../sample_testing/sample2.log')
dt.extractinfo(export=False,single_file=False)

In [ ]:
sg = Segmentation(0.3)
comp ='GPS'
va = 'RAlt'
fitmethod= 'inter'
sgmethod = 'td'
sr = dt.dbconnector.query(comp+'_sample_testing/sample2',va)
rng = dt.dbconnector.query(comp+'_sample_testing/sample2','lineIndex')
sg.segment(preprocessing.scale(sr),seg_method=sgmethod,fit_method=fitmethod,save_seginstance=False)
plt.figure(figsize=(12,6))
sg.draw_plot(comp+'_'+va)
sg.draw_segments()
plt.show()
#plt.savefig('../report/phase3/'+comp+'_'+va+'_'+sgmethod+'_'+fitmethod+'.png', dpi=500)

In [2]:
datalist =os.listdir('../data')
#added to the 100 variables
vardict = load_top100_dict()
vardict['RCOU'] = ['RCOU_Ch1','RCOU_Ch2','RCOU_Ccmdh3','RCOU_Ch4']

In [ ]:
dbc = DatabaseConnector('segdb','27017')
loader = DataLoader()
sg = Segmentation(1)
fitmethod= 'inter'
sgmethod = 'bu'
for i,log in enumerate(datalist):
    start_time = time.time()
    print('File:{} --  {} / {}'.format(log,i,len(datalist)-1))
    for k,v in vardict.items():
        print('Processing Component: {}..'.format(k))
        res = loader.dbconnector.query(k+'_'+log[:-4]) #find signals in the log files 
        if res != -1: #data exists
                df = pd.DataFrame(res)
                if len(df) < 5:
                    continue
                #check for each value if it is stationary or not
                for val in list(set(df.columns).intersection(v)): #loop through the signals
                    if not np.issubdtype(df[val].dtype, np.number):
                            cat_to_int(df,val)
                    list_segments = sg.segment(preprocessing.scale(df[val]),seg_method=sgmethod,fit_method=fitmethod,save_seginstance=False)
                    if isinstance(list_segments,int):
                        continue
                    #export list_segments into a collection
                    dbc.set_collection('seg_'+k+'_'+val+'_'+log[:-4])
                    resp = dbc.insert_dict(pd.DataFrame(list_segments,columns=['x1','y1','x2','y2']).to_dict('list'))
                    if resp == -1:
                        print('Dict insert error in file : {}'.format(log[:-4]))
    print('File Run Time: ', time.time() - start_time)
dbc.close_connection()

mongodb://localhost:27017/
Database connected
mongodb://localhost:27017/
Database connected
File:568c058bb560505c22dc402c.log --  0 / 2607
Processing Component: MODE..
Processing Component: GPS..
Processing Component: DU32..
Processing Component: ATT..
